In [1]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

# LeNet5 - Cat-Vs-Dog

In [2]:
import paths
import keras
from src.parameters import metrics
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.export.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.export.model_export import clear_exported_models
from src.keras_model.lenet5_network import LeNet5Network

Using TensorFlow backend.


In [3]:
network_name = 'lenet5'
fit_epochs = 50
dataset = 'cat-vs-dog'
dataset_percentage = '10'

In [4]:
network = LeNet5Network()

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    dataset_dir=paths.data_location,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for cat-vs-dog_10...
Found 1750 images belonging to 2 classes.
Loading test set for cat-vs-dog_10...
Found 500 images belonging to 2 classes.
Loading validation set for cat-vs-dog_10...
Found 250 images belonging to 2 classes.


In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
best_model_path = f'{checkpoint_path}best.h5'
clear_exported_models(checkpoint_path)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_accuracy}}.h5',
    save_best_only=False,
    save_weights_only=False
)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [checkpoint_callback, best_callback, time_callback]

In [8]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
55/55 [==============================] - 6s 110ms/step - loss: 1.2219 - accuracy: 0.5280 - precision: 0.5192 - recall: 0.5335 - val_loss: 0.7646 - val_accuracy: 0.5480 - val_precision: 0.5955 - val_recall: 0.2979
Epoch 2/50
55/55 [==============================] - 4s 67ms/step - loss: 0.6492 - accuracy: 0.6309 - precision: 0.6395 - recall: 0.6121 - val_loss: 0.5193 - val_accuracy: 0.6500 - val_precision: 0.6238 - val_recall: 0.7180
Epoch 3/50
55/55 [==============================] - 4s 67ms/step - loss: 0.5880 - accuracy: 0.6743 - precision: 0.6741 - recall: 0.7100 - val_loss: 0.6812 - val_accuracy: 0.6360 - val_precision: 0.6051 - val_recall: 0.7502
Epoch 4/50
55/55 [==============================] - 4s 67ms/step - loss: 0.5438 - accuracy: 0.7177 - precision: 0.7309 - recall: 0.7315 - val_loss: 0.8151 - val_accuracy: 0.6060 - val_precision: 0.5626 - val_recall: 0.8987
Epoch 5/50
55/55 [==============================] - 4s 64ms/step - loss: 0.5264 - accuracy: 0.7200 - precis

55/55 [==============================] - 4s 68ms/step - loss: 3.0353e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.3100 - val_accuracy: 0.6420 - val_precision: 0.6326 - val_recall: 0.6565
Epoch 38/50
55/55 [==============================] - 4s 67ms/step - loss: 2.8046e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.2155 - val_accuracy: 0.6420 - val_precision: 0.6369 - val_recall: 0.6733
Epoch 39/50
55/55 [==============================] - 4s 68ms/step - loss: 2.5222e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.1632 - val_accuracy: 0.6440 - val_precision: 0.6388 - val_recall: 0.6681
Epoch 40/50
55/55 [==============================] - 4s 68ms/step - loss: 2.2975e-04 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - val_loss: 3.4720 - val_accuracy: 0.6460 - val_precision: 0.6505 - val_recall: 0.6681
Epoch 41/50
55/55 [==============================] - 4s 68ms/step - loss: 2.1549e-04 - accuracy: 1.0

In [9]:
best_model = keras.models.load_model(best_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)
validation_result = best_model.evaluate_generator(
    generator=dataset_generators.validate
)

## Results

### Epoch CSV

In [10]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

epoch, time, val_loss, val_accuracy, val_precision, val_recall
0,0,0,0,0,0
1,6.0639259815216064,0.7646225690841675,0.5479999780654907,0.5955109000205994,0.29789918661117554
2,3.730588912963867,0.5193068981170654,0.6499999761581421,0.6237736940383911,0.7180382013320923
3,3.6839990615844727,0.6811953186988831,0.6359999775886536,0.6050700545310974,0.750240683555603
4,3.732018232345581,0.8150625228881836,0.6060000061988831,0.5626401901245117,0.8986744284629822
5,3.5343921184539795,0.4265148639678955,0.6679999828338623,0.687727689743042,0.6037598252296448
6,3.5147128105163574,1.1622717380523682,0.6259999871253967,0.6785646677017212,0.45164617896080017
7,3.5180869102478027,0.6720091700553894,0.6380000114440918,0.5982322692871094,0.8941740989685059
8,3.4975061416625977,1.1835198402404785,0.6800000071525574,0.6629409193992615,0.720952570438385
9,3.512525796890259,0.5851806998252869,0.6520000100135803,0.653448760509491,0.6563054919242859
10,3.6014249324798584,0.5696600675582886,0.65799999237060

### Validation result

In [11]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)

avg_epoch_time, time, val_loss, val_accuracy, val_precision, val_recall
3.7909540128707886,189.54770064353943,2.523925542831421,0.671999990940094,0.6482727527618408,0.7386161088943481
